# Identify proper nouns

In [138]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
import numpy as np
import seaborn as sns

In [164]:
newwords = pd.read_csv('docfreqs.tsv', sep = '\t', low_memory = False)
newwords.sort_values(by = 'authorcount', ascending = False, inplace = True)
newwords.reset_index(drop = True, inplace = True)
newwords.head()

,word,allcount,alldocs,capcount,capdocs,burstiness,capratio,capburstiness,authorcount
0,a,27030805,30682,0,0,880.995923,0.000000,0.000000,4945
1,s,9696488,24397,0,0,397.444286,0.000000,0.000000,4944
2,to,31509834,35009,342448,15226,900.046959,0.010868,22.490855,4942
3,in,18140861,36068,896707,15308,502.961370,0.049430,58.577289,4942
4,and,35791856,35519,2407284,15325,1007.679136,0.067258,157.081128,4942


In [165]:
newwords.shape

(2242945, 9)

In [145]:
proper = set()
with open('/Users/tunder/Dropbox/DataMunging/rulesets/PersonalNames.txt') as f:
    for line in f:
        proper.add(line.strip().lower())

#with open('/Users/tunder/Dropbox/DataMunging/rulesets/PlaceNames.txt') as f:
    #for line in f:
        #proper.add(line.strip().lower())

In [146]:
len(proper)

4644

In [166]:
keepthese = []
for idx, row in newwords.iterrows():
    if idx < 90000:
        res = True
    elif row['word'] in proper:
        res = True
    else:
        res = False
    keepthese.append(res)

newwords['tokeep'] = keepthese

In [167]:
words = newwords.loc[newwords.tokeep, : ]

In [168]:
words.shape

(92108, 10)

In [172]:
words.iloc[89900:90000, : ]

,word,allcount,alldocs,capcount,capdocs,burstiness,capratio,capburstiness,authorcount,tokeep,proper
89900,genug,77,38,27,14,2.020997,0.350195,1.914894,33,True,0
89901,murcia,154,42,153,41,3.657957,0.992862,3.722628,33,True,0
89902,balmier,38,37,1,1,1.024259,0.026247,0.909091,33,True,0
89903,morto,45,36,12,10,1.246537,0.266075,1.188119,33,True,0
89904,marrer,73,40,1,1,1.820449,0.013680,0.909091,33,True,0
...,...,...,...,...,...,...,...,...,...,...,...
89995,smiter,61,50,13,9,1.217565,0.212766,1.428571,33,True,0
89996,/4,68,37,0,0,1.832884,0.000000,0.000000,33,True,0
89997,larcenies,41,40,0,0,1.022444,0.000000,0.000000,33,True,0
89998,uut,47,41,19,17,1.143552,0.403397,1.111111,33,True,0


In [169]:
def properornot(word):
    global proper
    if word in proper:
        return 1
    else:
        return 0

words['proper'] = words.word.map(properornot)

/Users/tunder/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [170]:
model = LogisticRegression(C = 1000, class_weight = 'balanced', max_iter = 1000)
cv_results = cross_validate(model, X = words.iloc[ : , 1:9], y = words.iloc[ : , 10], cv = 10)
print(np.mean(cv_results['test_score']))

0.8849904807439435


In [173]:
model = LogisticRegression(C = 1000, class_weight = 'balanced', max_iter = 1000)
predictions = cross_val_predict(model, X = words.iloc[ : , 1:9], y = words.iloc[ : , 10], cv = 10, method = 'predict_proba')

In [174]:
len(predictions)

92108

In [175]:
predictions = predictions[ : , 1]

In [179]:
words['predicted'] = predictions

/Users/tunder/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [180]:
words.to_csv('modeled90k.tsv', sep = '\t', index = False)

In [185]:
words = pd.read_csv('modeled90k.tsv', sep = '\t', low_memory = False)

In [186]:
words.head()

,word,allcount,alldocs,capcount,capdocs,burstiness,capratio,capburstiness,authorcount,proper,predicted,confirmed
0,but,7428932,31848,2689188,15340,233.261388,0.361989,175.304463,4940,0,1.0,1.0
1,they,5751000,31978,1580117,15333,179.841829,0.274755,103.052677,4938,0,1.0,1.0
2,then,2922126,31049,945403,15305,94.113066,0.323533,61.770456,4937,0,1.0,1.0
3,he,20321925,33593,5992595,15329,604.943426,0.294883,390.929344,4936,0,1.0,1.0
4,well,1728748,31490,711788,15357,54.898143,0.411736,46.349115,4936,0,1.0,1.0


In [197]:
words = words.assign(capdocratio = np.log(words.alldocs / (words.capdocs + 0.1)))

In [188]:
words = words.assign(logauthor = np.log(words.authorcount))

In [198]:
words.iloc[340:360, : ]

,word,allcount,alldocs,capcount,capdocs,burstiness,capratio,capburstiness,authorcount,proper,predicted,confirmed,capdocratio,logauthor
340,pickwick,13500,312,13354,279,43.255367,0.989178,47.846650,198,1,0.995853,1.0,0.111433,5.288267
341,sewell,2406,112,2369,100,21.462979,0.984581,23.666334,90,1,0.995840,1.0,0.112329,4.499810
342,barron,2286,107,2268,100,21.344538,0.992083,22.657343,93,1,0.995829,1.0,0.066659,4.532599
343,alain,2740,129,2736,125,21.223857,0.998504,21.870504,105,1,0.995817,1.0,0.030699,4.653960
344,kenyon,2050,97,2047,95,21.112255,0.998488,21.524711,79,1,0.995768,1.0,0.019782,4.369448
345,loring,3191,141,3187,137,22.615167,0.998715,23.245806,125,1,0.995744,1.0,0.028049,4.828314
346,orme,1833,63,1827,58,29.049128,0.996672,31.445783,52,1,0.995695,1.0,0.080969,3.951244
347,galt,1582,48,1566,38,32.889813,0.989824,41.102362,39,1,0.995685,1.0,0.230987,3.663562
348,bama,1401,42,1380,30,33.277910,0.984940,45.847176,37,1,0.995597,1.0,0.333144,3.610918
349,lassiter,1239,39,1238,38,31.687980,0.999112,32.493438,37,1,0.995514,1.0,0.023347,3.610918


In [199]:
X = words.loc[ : , ['allcount', 'alldocs', 'capcount', 'capdocs', 'burstiness', 'capratio', 'capburstiness', 'authorcount', 'capdocratio', 'logauthor']]

In [200]:
y = words['proper']

In [202]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [206]:
model = LogisticRegression(C = 10000, class_weight = 'balanced', max_iter = 1000)
cv_results = cross_validate(model, X = X, y = y, cv = 10)
print(np.mean(cv_results['test_score']))

0.9076644539745061


In [208]:
model = LogisticRegression(C = 1000, class_weight = 'balanced', max_iter = 1000)
predictions = cross_val_predict(model, X = X, y = y, cv = 10, method = 'predict_proba')

In [209]:
probabilities = predictions[ : , 1]

def binarize(afloat):
    if afloat > 0.5:
        return 1
    else:
        return 0

predictions  = [binarize(x) for x in probabilities]

In [210]:
words = words.assign(predicted = predictions)
words = words.assign(probs = probabilities)

In [213]:
words.to_csv('remodeled90k.tsv', sep = '\t', index = False)

In [214]:
rewords = pd.read_csv('remodeled90k.tsv', sep = '\t', low_memory = False)

In [215]:
def tokeep(row):
    if row['predicted'] == 0 and row['authorcount'] > 34 and row['proper'] == 0:
        return True
    elif row['proper'] == 0 and row['confirmed'] == 1:
        return True
    else:
        return False

kept = []

for idx, row in rewords.iterrows():
    kept.append(tokeep(row))


In [216]:
sum(kept)

78553

In [217]:
rewords = rewords.assign(kept = kept)

In [220]:
vocabulary = rewords.loc[rewords.kept == True, 'word']

In [222]:
propernames = rewords.loc[(rewords.kept == False) & (rewords.authorcount > 34), 'word']

In [226]:
inlist = "‘’"
outlist = "''"
removallist = '“”"'
translator = str.maketrans(inlist, outlist, removallist)

atest = 'don‘t use "quotations in you”re prose'
atest.translate(translator)

"don't use quotations in youre prose"

In [238]:
# vocabulary = vocabulary.tolist()
vocab = set([x.translate(translator) for x in vocabulary if type(x) == str])
len(vocab)

78521

In [240]:
vocab = list(vocab)

In [241]:
vocab[0:10]

['bathrooms',
 'outpatient',
 'obbligato',
 'i69',
 'bridle',
 '43rd',
 'pomposity',
 'whoof',
 'exempting',
 'petalled']

In [255]:
vocabulary = []
wordset = set(rewords.word.tolist())
for v in vocab:
    if v in wordset:
        authors = rewords.loc[rewords.word == v, 'authorcount'].values[0]
        vocabulary.append((int(authors), v))
    else:
        vocabulary.append((0, v))
        

In [256]:
vocabulary.sort(reverse = True)
vocabulary[0:10]

[(4945, 'a'),
 (4944, 's'),
 (4942, 'to'),
 (4942, 'on'),
 (4942, 'in'),
 (4942, 'and'),
 (4941, 'the'),
 (4941, 'of'),
 (4941, 'it'),
 (4941, 'is')]

In [259]:
with open('modelvocab.tsv', mode = 'w', encoding = 'utf-8') as f:
    for authorcount, word in vocabulary:
        if authorcount < 35:
            continue
        elif '&quot' in word or 'ﬁ' in word or 'ﬂ' in word:
            continue
        else:
            f.write(word + '\t' + str(authorcount) + '\n')

In [261]:
with open('modelvocab.tsv', mode = 'r', encoding = 'utf-8') as f:
    lines = f.readlines()

with open('modelvocab.tsv', mode = 'w', encoding = 'utf-8') as f:
    for l in lines:
        if '_' in l:
            continue
        if ')' in l:
            continue
        if '^' in l:
            continue
        if '>' in l:
            continue
        if '<' in l:
            continue
        if '}' in l:
            continue
        if '/' in l:
            continue
        if '■' in l:
            continue
        if ',' in l:
            continue
        if ';' in l:
            continue
        if '—' in l:
            continue
        
        f.write(l)

In [263]:
import csv

translator = dict()

with open('CorrectionRules.txt', encoding = 'utf-8') as f:
    reader = csv.reader(f, delimiter = '\t')
    for row in reader:
        if len(row) < 2:
            continue
        translator[row[0]] = row[1]

with open('VariantSpellings.txt', encoding = 'utf-8') as f:
    reader = csv.reader(f, delimiter = '\t')
    for row in reader:
        if len(row) < 2:
            continue
        translator[row[0]] = row[1]

In [264]:
vocab = []
vocabset = set()
with open('modelvocab.tsv', mode = 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    for l in lines:
        l = l.strip()
        fields = l.split('\t')
        vocab.append((fields[0], fields[1]))
        vocabset.add(fields[0])       

In [281]:
translated = 0
untranslated = 0
newset = set()
with open('modelvocab2.tsv', mode = 'w', encoding = 'utf-8') as f:
    for word, authorcount in vocab:
        if word in translator and translator[word] in vocabset:
            translated += 1
            continue
        elif word in translator and ' ' in translator[word]:
            continue
        elif word in translator and translator[word] in newset:
            continue
        elif word in translator:
            word = translator[word]
            newset.add(word)
            untranslated += 1
        
        f.write(word + '\t' + str(authorcount) + '\n')
        
print(translated, untranslated)

1507 419


In [279]:
translator['everytime']

'every time'

In [269]:
ctr = 0
for k, v in translator.items():
    print(k, v)
    ctr += 1
    if ctr > 100:
        break

tbe the
foon soon
tlie the
vour your
lnternet internet
witli with
tliis this
lonization ionization
printf prints
1nternational international
thev they
l6pez lopez
lslam islam
majefty majesty
whicli which
lslamic islamic
g6mez gomez
fhe she
tliey they
1nstitute institute
avith with
lndonesia indonesia
f1g fig
ram6n ramon
tbat that
tlieir their
liave have
califomia california
avas was
montferrat montserrat
latrogenic iatrogenic
1ndian indian
hfe life
gaf gas
whieh which
veiy very
vhich which
0ther other
1ndia india
sucli such
avhen when
tbey they
ihey they
lieben lichen
lncreased increased
notwith not with
lndividual individual
tliem them
snch such
avhich which
wliat what
fex sex
0ctober october
tliere there
httle little
onlv only
ihat that
fects sects
1ndeed indeed
11linois illinois
iheir their
c6rdoba cordoba
verv very
heing being
witb with
trav tray
1nformation information
ufe use
1ntroduction introduction
eveiy every
tbis this
feated seated
intemet internet
tiiat that
calder6n calder

In [270]:
len(vocabset)

77022

In [271]:
'honor' in vocabset

True

In [272]:
'honour' in vocabset

True

In [273]:
'honor' in translator

True

In [282]:
j = 'unholy—one'
j.split('—')

['unholy', 'one']